In [25]:
# Import libraries
from pyspark.sql import *
from pyspark.sql.functions import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psutil
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time
import json

# Linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Random forest
from sklearn.ensemble import RandomForestRegressor

# Gaussian process
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
import torch
import gpytorch

# Neural network
from sklearn.neural_network import MLPRegressor

# Gradient boosting
from sklearn.ensemble import GradientBoostingRegressor

# Import own functions
import helperFuncs as hf
import metrics
import methods

In [ ]:
%matplotlib inline

In [32]:
%load_ext autoreload
%autoreload 2

In [2]:
# Create spark session
spark = SparkSession.builder.getOrCreate()

# Create spark context
sc = spark.sparkContext

# Check how much memory is allocated to spark runtime
mem_info = psutil.virtual_memory()
total_memory_gb = mem_info.total / (1024 * 1024 * 1024)
print("Total Memory Allocated to Spark Runtime:", int(total_memory_gb), "GB")

24/05/28 11:27:42 WARN Utils: Your hostname, CC-M133A-EU.local resolves to a loopback address: 127.0.0.1; using 10.19.209.184 instead (on interface en0)
24/05/28 11:27:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 11:27:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Total Memory Allocated to Spark Runtime: 32 GB


In [3]:
# Load data
folder = '/Users/ekinokos2/Downloads/2023-citibike-tripdata/9_September/'
name = '202309-citibike-tripdata_4.csv'
path = folder + name

sep23_data = spark.read.csv(path, sep = ',', header = True, inferSchema = True)

In [5]:
sep23_data_pd = pd.read_csv(path)

/var/folders/84/204q7trx39741y4p1_tyrh6m0000gp/T/ipykernel_37832/1334132814.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sep23_data_pd = pd.read_csv(path)


In [4]:
# Choose which dataset
data = sep23_data

# View data
print(f"NUM DATA POINTS = {data.count()}")
data.show()

NUM DATA POINTS = 575162
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+-----------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|          end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+-----------------+------------------+-------------+
|E7CE7CE80519EB26| classic_bike|2023-09-16 17:15:28|2023-09-16 17:19:00|Halsey St & Ralph...|         4334.02|Broadway & Madiso...|       4483.10|          40.68494|         -73.92299|         40.68822|         -73.91966|       member|
|56F4C65B769CF128| classic_bike

In [5]:
# Project only relevant columns
projected_data = data.select("started_at", "ended_at", "start_lat", "start_lng",
                  "end_lat", "end_lng")

# Select a small sample (first N) of rows
N = 1000
small_data = projected_data.select("*").limit(N)

# Test function
test = hf.extract_times_to_seconds(small_data)
test.show()

+------------------+------------------+-----------------+------------------+-------------+-----------+
|         start_lat|         start_lng|          end_lat|           end_lng|start_seconds|end_seconds|
+------------------+------------------+-----------------+------------------+-------------+-----------+
|          40.68494|         -73.92299|         40.68822|         -73.91966|        62128|      62340|
|       40.73240447|     -73.998393893|        40.722055|        -73.989111|        78329|      78813|
| 40.78232260730881|-73.95987510681152|40.78866499127884|-73.96680057048798|        43198|      44582|
|40.750097682094825|-73.91349703073502|        40.756913|        -73.921631|        32947|      33267|
| 40.78232260730881|-73.95987510681152|40.78866499127884|-73.96680057048798|        74065|      74516|
| 40.78232260730881|-73.95987510681152|40.78866499127884|-73.96680057048798|        76446|      77789|
| 40.78232260730881|-73.95987510681152|40.78866499127884|-73.966800570487

In [6]:
# Search for best lambda and gamma
# Collect working dataset into an array
small_time_and_space = hf.extract_times_to_seconds(small_data)
small_time_and_space_arr = np.array(small_time_and_space.collect())

# Normalize the data
scaler = StandardScaler()
scaler.fit(small_time_and_space_arr)
small_time_and_space_arr = scaler.transform(small_time_and_space_arr)

In [28]:
lam_search = [0.9, 0.7, 0.5, 0.3, 0.1]
gam_search = [0.1, 0.5, 1.0, 1.5, 3.0]

for lam in lam_search:
    for gam in gam_search:
        # Set k, the number of clusters for K-means
        k = 5

        # Pick the first k points as a "random" initialization of centroids
        random_centroids = small_time_and_space_arr[:k]

        # Run k-means time & space algorithm on small_space_and_time data
        # lam (a hyper-parameter between 0 and 1) controls how much we weight space
        # vs. time in our calculation of the cost. The smaller lam is, the more space
        # is weighted.
        costs, centroids, clustering = hf.K_means_time_and_space(small_time_and_space_arr,
                                                            random_centroids,
                                                            max_iter = 10, norm = 2,
                                                            lam = lam,
                                                            gam = gam)
        # Plot the cost vs. iteration
        #plt.plot(costs)
        #plt.xlabel("Iteration")
        #plt.ylabel("Cost")
        #plt.title(f"Cost vs. Iteration for Lambda = {lam}, Gamma = {gam}")
        #plt.show()

        overall_score, sample_scores = hf.silhouette_score_custom(small_time_and_space_arr, clustering, hf.custom_distance_metric, lam, gam)
        print(f"Lambda = {lam}, Gamma = {gam}, Overall Silhouette  Score {overall_score}")

Lambda = 0.9, Gamma = 0.1, Overall Silhouette  Score 0.3729527852828081
Lambda = 0.9, Gamma = 0.5, Overall Silhouette  Score 0.37269928906311334
Lambda = 0.9, Gamma = 1.0, Overall Silhouette  Score 0.3723830184413008
Lambda = 0.9, Gamma = 1.5, Overall Silhouette  Score 0.37206741191487336
Lambda = 0.9, Gamma = 3.0, Overall Silhouette  Score 0.37112455366888436
Lambda = 0.7, Gamma = 0.1, Overall Silhouette  Score 0.38196284643434814
Lambda = 0.7, Gamma = 0.5, Overall Silhouette  Score 0.381694921300087
Lambda = 0.7, Gamma = 1.0, Overall Silhouette  Score 0.38136066916303785
Lambda = 0.7, Gamma = 1.5, Overall Silhouette  Score 0.3810271415335652
Lambda = 0.7, Gamma = 3.0, Overall Silhouette  Score 0.3800308794232987
Lambda = 0.5, Gamma = 0.1, Overall Silhouette  Score 0.38382632728703164
Lambda = 0.5, Gamma = 0.5, Overall Silhouette  Score 0.3835553559276871
Lambda = 0.5, Gamma = 1.0, Overall Silhouette  Score 0.3832173078205451
Lambda = 0.5, Gamma = 1.5, Overall Silhouette  Score 0.3828

## Outline of the remaining work
* Implement models to predict the arrival time using
    * Linear regression
    * Random forest
    * Gradient boosting
    * Neural network
    * Gaussian process
* Compare the performance of the models when run with the entire set of data and when run with the subset of data belonging to the specific cluster
* Compare our clustering approach with Euclidian distance clustering

In [7]:
# Project only relevant columns
projected_data = data.select("started_at", "ended_at", "start_lat", "start_lng",
                  "end_lat", "end_lng")

# Select a small sample (first N) of rows
N = 10000
small_data = projected_data.select("*").limit(N)

# Search for best lambda and gamma
# Collect working dataset into an array
small_time_and_space = hf.extract_times_to_seconds(small_data)
small_time_and_space_arr = np.array(small_time_and_space.collect())

# Normalize the data
scaler = StandardScaler()
scaler.fit(small_time_and_space_arr)
small_time_and_space_arr = scaler.transform(small_time_and_space_arr)

# Set k, the number of clusters for K-means
k = 5

lam = 0.5
gam = 1.0

# Pick the first k points as a "random" initialization of centroids
random_centroids = small_time_and_space_arr[:k]

# Run k-means time & space algorithm on small_space_and_time data
# lam (a hyper-parameter between 0 and 1) controls how much we weight space
# vs. time in our calculation of the cost. The smaller lam is, the more space
# is weighted.
costs, centroids, clustering = hf.K_means_time_and_space(small_time_and_space_arr,
                                                    random_centroids,
                                                    max_iter = 10, norm = 2,
                                                    lam = lam,
                                                    gam = gam)

In [8]:
# Add clustering to the dataframe
small_time_and_space_arr_pd = pd.DataFrame(small_time_and_space_arr)
small_time_and_space_arr_pd.columns = ["start_lat", "start_lng", "end_lat", "end_lng", "start_time", "end_time"]
small_time_and_space_arr_pd["cluster"] = clustering

# Assuming 'data' is the DataFrame containing the data
# and 'clustering' is the numpy array containing the cluster values

cluster_train_test_splits = hf.cluster_train_test_split(small_time_and_space_arr_pd)


# Also do a regular train/test split
X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
y = small_time_and_space_arr_pd["end_time"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
small_time_and_space_arr_pd["cluster"].value_counts()

1.0    2906
0.0    2783
2.0    1966
3.0    1176
4.0    1169
Name: cluster, dtype: int64

In [10]:
# Train a linear regression model for each cluster
linear_regression_models = {}
lr_cluster_times = {}
for cluster in cluster_train_test_splits:
    start = time.time()
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    model = LinearRegression()
    model.fit(train_x, train_y)
    linear_regression_models[cluster] = model
    end = time.time()
    lr_cluster_times[cluster] = end - start

# Evaluate the linear regression models
linear_regression_mse = {}
lr_cluster_results = {}
for cluster in linear_regression_models:
    model = linear_regression_models[cluster]
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    predictions = model.predict(test_x)
    mse = mean_squared_error(test_y, predictions)
    linear_regression_mse[cluster] = mse
    results = metrics.evaluate(test_y, predictions)
    lr_cluster_results[cluster] = results

# Also do a regular train/test split
X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
y = small_time_and_space_arr_pd["end_time"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model for the entire dataset
model = LinearRegression()
start = time.time()
model.fit(train_x, train_y)
end = time.time()
lr_time = end - start
predictions = model.predict(test_x)
lr_mse = mean_squared_error(test_y, predictions)
lr_entire_results = metrics.evaluate(test_y, predictions)

print(f"Training time for entire dataset: {lr_time}")
print()
print(f"Linear Regression MSE for Entire Dataset: {lr_mse}")
print()
print("Linear Regression Training Times for Each Cluster:")
for cluster in lr_cluster_times:
    print(f"Cluster {cluster}: {lr_cluster_times[cluster]}")
print()
print("Linear Regression MSE for Each Cluster:")
for cluster in linear_regression_mse:
    print(f"Cluster {cluster}: {linear_regression_mse[cluster]}")

# Print the average of all entries in lr_cluster_results
print()
print("Average of all entries in lr_cluster_results:")
lr_cluster_results_pd = pd.DataFrame(lr_cluster_results)
print(lr_cluster_results_pd.mean(axis=1))

Training time for entire dataset: 0.0012881755828857422

Linear Regression MSE for Entire Dataset: 0.12155805688854653

Linear Regression Training Times for Each Cluster:
Cluster 0.0: 0.0018339157104492188
Cluster 1.0: 0.0020568370819091797
Cluster 2.0: 0.0009860992431640625
Cluster 3.0: 0.0012049674987792969
Cluster 4.0: 0.0007288455963134766

Linear Regression MSE for Each Cluster:
Cluster 0.0: 0.039213024809092306
Cluster 1.0: 0.14505166299762787
Cluster 2.0: 0.003318368760180141
Cluster 3.0: 0.01796160488738441
Cluster 4.0: 0.1765149867168799

Average of all entries in lr_cluster_results:
MAE        0.069936
RMSE       0.238128
MAD        0.045427
MAPE       0.252623
MAXAPE    33.926156
TAPE      97.099360
dtype: float64


In [11]:
lr_entire_results

{'MAE': 0.0711493164406142,
 'RMSE': 0.34865177023578486,
 'MAD': 0.037222118158097006,
 'MAPE': 0.17124203540145422,
 'MAXAPE': 18.954042224448255,
 'TAPE': 342.48407080290843}

In [14]:
# Train a random forest model for each cluster
random_forest_models = {}
rf_cluster_times = {}
for cluster in cluster_train_test_splits:
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    model = RandomForestRegressor()
    start = time.time()
    model.fit(train_x, train_y)
    end = time.time()
    rf_cluster_times[cluster] = end - start
    random_forest_models[cluster] = model

# Evaluate the random forest models
random_forest_mse = {}
for cluster in random_forest_models:
    model = random_forest_models[cluster]
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    predictions = model.predict(test_x)
    mse = mean_squared_error(test_y, predictions)
    random_forest_mse[cluster] = mse
    
# Also do a regular train/test split
X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
y = small_time_and_space_arr_pd["end_time"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest model for the entire dataset
model = RandomForestRegressor()
start = time.time()
model.fit(train_x, train_y)
end = time.time()
rf_time = end - start
predictions = model.predict(test_x)
mse = mean_squared_error(test_y, predictions)

print(f"Training Time for Entire Dataset: {rf_time}")
print()
print(f"Random Forest MSE for Entire Dataset: {mse}")
print()
print("Random Forest Training Times for Each Cluster:")
for cluster in rf_cluster_times:
    print(f"Cluster {cluster}: {rf_cluster_times[cluster]}")
print()
print("Random Forest MSE for Each Cluster:")
for cluster in random_forest_mse:
    print(f"Cluster {cluster}: {random_forest_mse[cluster]}")

Training Time for Entire Dataset: 2.2465171813964844

Random Forest MSE for Entire Dataset: 0.043384756654168626

Random Forest Training Times for Each Cluster:
Cluster 0.0: 0.5286421775817871
Cluster 1.0: 0.5370738506317139
Cluster 2.0: 0.35738706588745117
Cluster 3.0: 0.21899890899658203
Cluster 4.0: 0.21520614624023438

Random Forest MSE for Each Cluster:
Cluster 0.0: 0.02144629111257377
Cluster 1.0: 0.03772802743462243
Cluster 2.0: 0.005291408981845475
Cluster 3.0: 0.08192402372832287
Cluster 4.0: 0.15909367869171734


In [15]:
# Train a neural network model for each cluster
neural_network_models = {}
nn_cluster_times = {}
for cluster in cluster_train_test_splits:
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    model = MLPRegressor()
    start = time.time()
    model.fit(train_x, train_y)
    end = time.time()
    nn_cluster_times[cluster] = end - start
    neural_network_models[cluster] = model

# Evaluate the neural network models
neural_network_mse = {}
for cluster in neural_network_models:
    model = neural_network_models[cluster]
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    predictions = model.predict(test_x)
    mse = mean_squared_error(test_y, predictions)
    neural_network_mse[cluster] = mse

# Also do a regular train/test split
X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
y = small_time_and_space_arr_pd["end_time"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a neural network model for the entire dataset
model = MLPRegressor()
start = time.time()
model.fit(train_x, train_y)
end = time.time()
nn_time = end - start
predictions = model.predict(test_x)
mse = mean_squared_error(test_y, predictions)

print(f"NN Training Time for Entire Dataset: {nn_time}")
print()
print(f"NN MSE for Entire Dataset: {mse}")
print()
print("NN Training Times for Each Cluster:")
for cluster in nn_cluster_times:
    print(f"Cluster {cluster}: {nn_cluster_times[cluster]}")
print()
print("NN MSE for Each Cluster:")
for cluster in neural_network_mse:
    print(f"Cluster {cluster}: {neural_network_mse[cluster]}")

/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


NN Training Time for Entire Dataset: 2.6895737648010254

NN MSE for Entire Dataset: 0.08650308140077245

NN Training Times for Each Cluster:
Cluster 0.0: 0.7068591117858887
Cluster 1.0: 0.7510719299316406
Cluster 2.0: 0.1628129482269287
Cluster 3.0: 0.3173229694366455
Cluster 4.0: 0.09429192543029785

NN MSE for Each Cluster:
Cluster 0.0: 0.036605677170569006
Cluster 1.0: 0.1173815393100648
Cluster 2.0: 0.007457718034959324
Cluster 3.0: 0.026199237086368052
Cluster 4.0: 0.17337422702407582


/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [17]:
num_epochs = 25
gp_lr = 0.1
    
# Train a Gaussian process model for each cluster
gaussian_process_models = {}
gp_cluster_times = {}
for cluster in cluster_train_test_splits:
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    train_x = torch.tensor(train_x.values).float()
    train_y = torch.tensor(train_y.values).float()
    likelihood = GaussianLikelihood()
    model = methods.GPModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()
    mll = ExactMarginalLogLikelihood(likelihood, model)
    optimizer = torch.optim.Adam(model.parameters(), lr=gp_lr)
    start = time.time()
    for i in range(num_epochs):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()
    end = time.time()
    gp_cluster_times[cluster] = end - start
    gaussian_process_models[cluster] = model

# Evaluate the Gaussian process models
gaussian_process_mse = {}
for cluster in gaussian_process_models:
    model = gaussian_process_models[cluster]
    test_x, test_y = cluster_train_test_splits[cluster][1], cluster_train_test_splits[cluster][3]
    test_x = torch.tensor(test_x.values).float()
    test_y = torch.tensor(test_y.values).float()
    model.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        predictions = model(test_x)
        mse = mean_squared_error(test_y, predictions.mean)
        gaussian_process_mse[cluster] = mse

# Also do a regular train/test split
X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
y = small_time_and_space_arr_pd["end_time"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gaussian process model for the entire dataset
train_x = torch.tensor(train_x.values).float()
train_y = torch.tensor(train_y.values).float()
likelihood = GaussianLikelihood()
model = methods.GPModel(train_x, train_y, likelihood)
model.train()
likelihood.train()
mll = ExactMarginalLogLikelihood(likelihood, model)
optimizer = torch.optim.Adam(model.parameters(), lr=gp_lr)
start = time.time()
for i in range(num_epochs):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()
end = time.time()
gp_time = end - start
test_x = torch.tensor(test_x.values).float()
test_y = torch.tensor(test_y.values).float()
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = model(test_x)
    mse = mean_squared_error(test_y, predictions.mean)

print(f"GP Training Time for Entire Dataset: {gp_time}")
print()
print(f"GP MSE for Entire Dataset: {mse}")
print()
print("GP Training Times for Each Cluster:")
for cluster in gp_cluster_times:
    print(f"Cluster {cluster}: {gp_cluster_times[cluster]}")
print()
print("GP MSE for Each Cluster:")
for cluster in gaussian_process_mse:
    print(f"Cluster {cluster}: {gaussian_process_mse[cluster]}")


GP Training Time for Entire Dataset: 12.097362041473389

GP MSE for Entire Dataset: 0.06379331648349762

GP Training Times for Each Cluster:
Cluster 0.0: 1.7533986568450928
Cluster 1.0: 1.734299898147583
Cluster 2.0: 0.6195230484008789
Cluster 3.0: 0.3039207458496094
Cluster 4.0: 0.30571484565734863

GP MSE for Each Cluster:
Cluster 0.0: 0.03782009333372116
Cluster 1.0: 0.06882806122303009
Cluster 2.0: 0.009221695363521576
Cluster 3.0: 0.043596670031547546
Cluster 4.0: 0.1631726324558258


In [18]:
# Train a gradient boosting model for each cluster
gradient_boosting_models = {}
gb_cluster_times = {}
for cluster in cluster_train_test_splits:
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    model = GradientBoostingRegressor()
    start = time.time()
    model.fit(train_x, train_y)
    end = time.time()
    gb_cluster_times[cluster] = end - start
    gradient_boosting_models[cluster] = model

# Evaluate the gradient boosting models
gradient_boosting_mse = {}
for cluster in gradient_boosting_models:
    model = gradient_boosting_models[cluster]
    train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
    predictions = model.predict(test_x)
    mse = mean_squared_error(test_y, predictions)
    gradient_boosting_mse[cluster] = mse

# Also do a regular train/test split
X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
y = small_time_and_space_arr_pd["end_time"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a gradient boosting model for the entire dataset
model = GradientBoostingRegressor()
start = time.time()
model.fit(train_x, train_y)
end = time.time()
gb_time = end - start
predictions = model.predict(test_x)
mse = mean_squared_error(test_y, predictions)

print(f"Training time for entire dataset: {gb_time}")
print()
print(f"Gradient Boosting MSE for Entire Dataset: {mse}")
print()
print("Gradient Boosting Training Times for Each Cluster:")
for cluster in gb_cluster_times:
    print(f"Cluster {cluster}: {gb_cluster_times[cluster]}")
print()
print("Gradient Boosting MSE for Each Cluster:")
for cluster in gradient_boosting_mse:
    print(f"Cluster {cluster}: {gradient_boosting_mse[cluster]}")

Training time for entire dataset: 0.8310761451721191

Gradient Boosting MSE for Entire Dataset: 0.03814455236191411

Gradient Boosting Training Times for Each Cluster:
Cluster 0.0: 0.22678589820861816
Cluster 1.0: 0.2177867889404297
Cluster 2.0: 0.15129613876342773
Cluster 3.0: 0.09425926208496094
Cluster 4.0: 0.09214210510253906

Gradient Boosting MSE for Each Cluster:
Cluster 0.0: 0.026964058798025135
Cluster 1.0: 0.037025286744018095
Cluster 2.0: 0.006013536929567091
Cluster 3.0: 0.09805893359151696
Cluster 4.0: 0.1755063411688478


In [65]:
# List of all parameters for sensitivity analysis
# [k, lam, gam, N, norm]
N = [1000, 10000, 100000]
norms = [1, 2]
lams = [0.1, 0.5, 0.9]
gams = [0.5, 1.0, 2.0]
K = [3, 6, 9]
test_size = 0.2
random_state = 42
max_iters = 10
num_epochs = 25
gp_lr = 0.1

verbose = True

# Run sensitivity analysis
for n in N:
    for norm in norms:
        for lam in lams:
            for gam in gams:
                for k in K:
                    print(f"N = {n}, Norm = {norm}, Lambda = {lam}, Gamma = {gam}, K = {k}")
                    print()
                    
                    # Project only relevant columns
                    projected_data = data.select("started_at", "ended_at", "start_lat", "start_lng",
                                    "end_lat", "end_lng")
                    
                    # Select a small sample (first N) of rows
                    small_data = projected_data.select("*").limit(n)

                    # Collect working dataset into an array
                    small_time_and_space = hf.extract_times_to_seconds(small_data)
                    small_time_and_space_arr = np.array(small_time_and_space.collect())

                    # Normalize the data
                    scaler = StandardScaler()
                    scaler.fit(small_time_and_space_arr)
                    small_time_and_space_arr = scaler.transform(small_time_and_space_arr)

                    # Pick the first k points as a "random" initialization of centroids
                    random_centroids = small_time_and_space_arr[:k]

                    # Run k-means time & space algorithm on small_space_and_time data
                    # lam (a hyper-parameter between 0 and 1) controls how much we weight space
                    # vs. time in our calculation of the cost. The smaller lam is, the more space
                    # is weighted.
                    costs, centroids, clustering = hf.K_means_time_and_space(small_time_and_space_arr,
                                                                        random_centroids,
                                                                        max_iter = max_iters, norm = norm,
                                                                        lam = lam,
                                                                        gam = gam)
                    
                    # Add clustering to the dataframe
                    small_time_and_space_arr_pd = pd.DataFrame(small_time_and_space_arr)
                    small_time_and_space_arr_pd.columns = ["start_lat", "start_lng", "end_lat", "end_lng", "start_time", "end_time"]
                    small_time_and_space_arr_pd["cluster"] = clustering

                    cluster_train_test_splits = hf.cluster_train_test_split(small_time_and_space_arr_pd)

                    # Linear regression
                    linear_regression_models = {}
                    lr_cluster_times = {}
                    for cluster in cluster_train_test_splits:
                        start = time.time()
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        model = LinearRegression()
                        model.fit(train_x, train_y)
                        linear_regression_models[cluster] = model
                        end = time.time()
                        lr_cluster_times[cluster] = end - start

                    # Evaluate the linear regression models
                    linear_regression_mse = {}
                    lr_cluster_results = {}
                    for cluster in linear_regression_models:
                        model = linear_regression_models[cluster]
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        predictions = model.predict(test_x)
                        lr_cluster_mse = mean_squared_error(test_y, predictions)
                        linear_regression_mse[cluster] = lr_cluster_mse
                        results = metrics.evaluate(test_y, predictions)
                        lr_cluster_results[cluster] = results

                    # Also do a regular train/test split
                    X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
                    y = small_time_and_space_arr_pd["end_time"]
                    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=random_state)

                    # Train a linear regression model for the entire dataset
                    model = LinearRegression()
                    start = time.time()
                    model.fit(train_x, train_y)
                    end = time.time()
                    lr_time = end - start
                    predictions = model.predict(test_x)
                    lr_entire_mse = mean_squared_error(test_y, predictions)
                    lr_entire_results = metrics.evaluate(test_y, predictions)

                    if verbose:
                        print(f"Training time for entire dataset: {lr_time}")
                        print()
                        print(f"Linear Regression MSE for Entire Dataset: {lr_entire_mse}")
                        print()
                        print("Linear Regression Training Times for Each Cluster:")
                        for cluster in lr_cluster_times:
                            print(f"Cluster {cluster}: {lr_cluster_times[cluster]}")
                        print()
                        print("Linear Regression MSE for Each Cluster:")
                        for cluster in linear_regression_mse:
                            print(f"Cluster {cluster}: {linear_regression_mse[cluster]}")

                        # Print the average of all entries in lr_cluster_results
                        print()
                        print("Average of all entries in lr_cluster_results:")
                        lr_cluster_results_pd = pd.DataFrame(lr_cluster_results)
                        print(lr_cluster_results_pd.mean(axis=1))
                        print()
                        print(f"Linear Regression Entire Dataset: {lr_entire_results}")

                    # Random forest
                    random_forest_models = {}
                    rf_cluster_times = {}
                    for cluster in cluster_train_test_splits:
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        model = RandomForestRegressor()
                        start = time.time()
                        model.fit(train_x, train_y)
                        end = time.time()
                        rf_cluster_times[cluster] = end - start
                        random_forest_models[cluster] = model

                    random_forest_mse = {}
                    rf_cluster_results = {}
                    for cluster in random_forest_models:
                        model = random_forest_models[cluster]
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        predictions = model.predict(test_x)
                        rf_cluster_mse = mean_squared_error(test_y, predictions)
                        random_forest_mse[cluster] = rf_cluster_mse
                        results = metrics.evaluate(test_y, predictions)
                        rf_cluster_results[cluster] = results

                    X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
                    y = small_time_and_space_arr_pd["end_time"]
                    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=random_state)

                    model = RandomForestRegressor()
                    start = time.time()
                    model.fit(train_x, train_y)
                    end = time.time()
                    rf_time = end - start
                    predictions = model.predict(test_x)
                    rf_entire_mse = mean_squared_error(test_y, predictions)
                    rf_entire_results = metrics.evaluate(test_y, predictions)

                    if verbose:
                        print(f"Training Time for Entire Dataset: {rf_time}")
                        print()
                        print(f"Random Forest MSE for Entire Dataset: {rf_entire_mse}")
                        print()
                        print("Random Forest Training Times for Each Cluster:")
                        for cluster in rf_cluster_times:
                            print(f"Cluster {cluster}: {rf_cluster_times[cluster]}")
                        print()
                        print("Random Forest MSE for Each Cluster:")
                        for cluster in random_forest_mse:
                            print(f"Cluster {cluster}: {random_forest_mse[cluster]}")

                        # Print the average of all entries in rf_cluster_results
                        print()
                        print("Average of all entries in rf_cluster_results:")
                        rf_cluster_results_pd = pd.DataFrame(rf_cluster_results)
                        print(rf_cluster_results_pd.mean(axis=1))
                        print()
                        print(f"Random Forest Entire Dataset: {rf_entire_results}")

                    # Neural network
                    neural_network_models = {}
                    nn_cluster_times = {}
                    for cluster in cluster_train_test_splits:
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        model = MLPRegressor()
                        start = time.time()
                        model.fit(train_x, train_y)
                        end = time.time()
                        nn_cluster_times[cluster] = end - start
                        neural_network_models[cluster] = model
                    
                    neural_network_mse = {}
                    nn_cluster_results = {}
                    for cluster in neural_network_models:
                        model = neural_network_models[cluster]
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        predictions = model.predict(test_x)
                        nn_cluster_mse = mean_squared_error(test_y, predictions)
                        neural_network_mse[cluster] = nn_cluster_mse
                        results = metrics.evaluate(test_y, predictions)
                        nn_cluster_results[cluster] = results

                    X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
                    y = small_time_and_space_arr_pd["end_time"]
                    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=random_state)

                    model = MLPRegressor()
                    start = time.time()
                    model.fit(train_x, train_y)
                    end = time.time()
                    nn_time = end - start
                    predictions = model.predict(test_x)
                    nn_mse = mean_squared_error(test_y, predictions)
                    nn_entire_results = metrics.evaluate(test_y, predictions)

                    if verbose:
                        print(f"NN Training Time for Entire Dataset: {nn_time}")
                        print()
                        print(f"NN MSE for Entire Dataset: {nn_mse}")
                        print()
                        print("NN Training Times for Each Cluster:")
                        for cluster in nn_cluster_times:
                            print(f"Cluster {cluster}: {nn_cluster_times[cluster]}")
                        print()
                        print("NN MSE for Each Cluster:")
                        for cluster in neural_network_mse:
                            print(f"Cluster {cluster}: {neural_network_mse[cluster]}")

                        # Print the average of all entries in nn_cluster_results
                        print()
                        print("Average of all entries in nn_cluster_results:")
                        nn_cluster_results_pd = pd.DataFrame(nn_cluster_results)
                        print(nn_cluster_results_pd.mean(axis=1))
                        print()
                        print(f"NN Entire Dataset: {nn_entire_results}")

                    # Gaussian process
                    gaussian_process_models = {}
                    gp_cluster_times = {}
                    for cluster in cluster_train_test_splits:
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        train_x = torch.tensor(train_x.values).float()
                        train_y = torch.tensor(train_y.values).float()
                        likelihood = GaussianLikelihood()
                        model = methods.GPModel(train_x, train_y, likelihood)
                        model.train()
                        likelihood.train()
                        mll = ExactMarginalLogLikelihood(likelihood, model)
                        optimizer = torch.optim.Adam(model.parameters(), lr=gp_lr)
                        start = time.time()
                        for i in range(num_epochs):
                            optimizer.zero_grad()
                            output = model(train_x)
                            loss = -mll(output, train_y)
                            loss.backward()
                            optimizer.step()
                        end = time.time()
                        gp_cluster_times[cluster] = end - start
                        gaussian_process_models[cluster] = model

                    gaussian_process_mse = {}
                    gp_cluster_results = {}
                    for cluster in gaussian_process_models:
                        model = gaussian_process_models[cluster]
                        test_x, test_y = cluster_train_test_splits[cluster][1], cluster_train_test_splits[cluster][3]
                        test_x = torch.tensor(test_x.values).float()
                        test_y = torch.tensor(test_y.values).float()
                        model.eval()
                        likelihood.eval()
                        with torch.no_grad(), gpytorch.settings.fast_pred_var():
                            predictions = model(test_x)
                            gp_cluster_mse = mean_squared_error(test_y, predictions.mean)
                            gaussian_process_mse[cluster] = gp_cluster_mse
                            results = metrics.evaluate(test_y, predictions.mean)
                            gp_cluster_results[cluster] = results

                    X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
                    y = small_time_and_space_arr_pd["end_time"]
                    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=random_state)

                    train_x = torch.tensor(train_x.values).float()
                    train_y = torch.tensor(train_y.values).float()
                    likelihood = GaussianLikelihood()
                    model = methods.GPModel(train_x, train_y, likelihood)
                    model.train()
                    likelihood.train()
                    mll = ExactMarginalLogLikelihood(likelihood, model)
                    optimizer = torch.optim.Adam(model.parameters(), lr=gp_lr)
                    start = time.time()
                    for i in range(num_epochs):
                        optimizer.zero_grad()
                        output = model(train_x)
                        loss = -mll(output, train_y)
                        loss.backward()
                        optimizer.step()
                    end = time.time()
                    gp_time = end - start
                    test_x = torch.tensor(test_x.values).float()
                    test_y = torch.tensor(test_y.values).float()
                    model.eval()
                    likelihood.eval()
                    with torch.no_grad(), gpytorch.settings.fast_pred_var():
                        predictions = model(test_x)
                        gp_entire_mse = mean_squared_error(test_y, predictions.mean)
                        gp_entire_results = metrics.evaluate(test_y, predictions.mean)

                    if verbose:
                        print(f"GP Training Time for Entire Dataset: {gp_time}")
                        print()
                        print(f"GP MSE for Entire Dataset: {gp_entire_mse}")
                        print()
                        print("GP Training Times for Each Cluster:")
                        for cluster in gp_cluster_times:
                            print(f"Cluster {cluster}: {gp_cluster_times[cluster]}")
                        print()
                        print("GP MSE for Each Cluster:")
                        for cluster in gaussian_process_mse:
                            print(f"Cluster {cluster}: {gaussian_process_mse[cluster]}")
                        
                        # Print the average of all entries in nn_cluster_results
                        print()
                        print("Average of all entries in gp_cluster_results:")
                        gp_cluster_results_pd = pd.DataFrame(gp_cluster_results)
                        print(gp_cluster_results_pd.mean(axis=1))
                        print()
                        print(f"GP Entire Dataset: {gp_entire_results}")

                    # Gradient boosting
                    gradient_boosting_models = {}
                    gb_cluster_times = {}
                    for cluster in cluster_train_test_splits:
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        model = GradientBoostingRegressor()
                        start = time.time()
                        model.fit(train_x, train_y)
                        end = time.time()
                        gb_cluster_times[cluster] = end - start
                        gradient_boosting_models[cluster] = model

                    gradient_boosting_mse = {}
                    gb_cluster_results = {}
                    for cluster in gradient_boosting_models:
                        model = gradient_boosting_models[cluster]
                        train_x, test_x, train_y, test_y = cluster_train_test_splits[cluster]
                        predictions = model.predict(test_x)
                        gb_cluster_mse = mean_squared_error(test_y, predictions)
                        gradient_boosting_mse[cluster] = gb_cluster_mse
                        results = metrics.evaluate(test_y, predictions)
                        gb_cluster_results[cluster] = results

                    X = small_time_and_space_arr_pd.drop(columns=["cluster", "end_time"])
                    y = small_time_and_space_arr_pd["end_time"]
                    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=random_state)

                    model = GradientBoostingRegressor()
                    start = time.time()
                    model.fit(train_x, train_y)
                    end = time.time()
                    gb_time = end - start
                    predictions = model.predict(test_x)
                    gb_entire_mse = mean_squared_error(test_y, predictions)
                    gb_entire_results = metrics.evaluate(test_y, predictions)

                    if verbose:
                        print(f"Training time for entire dataset: {gb_time}")
                        print()
                        print(f"Gradient Boosting MSE for Entire Dataset: {gb_entire_mse}")
                        print()
                        print("Gradient Boosting Training Times for Each Cluster:")
                        for cluster in gb_cluster_times:
                            print(f"Cluster {cluster}: {gb_cluster_times[cluster]}")
                        print()
                        print("Gradient Boosting MSE for Each Cluster:")
                        for cluster in gradient_boosting_mse:
                            print(f"Cluster {cluster}: {gradient_boosting_mse[cluster]}")

                        # Print the average of all entries in gb_cluster_results
                        print()
                        print("Average of all entries in gb_cluster_results:")
                        gb_cluster_results_pd = pd.DataFrame(gb_cluster_results)
                        print(gb_cluster_results_pd.mean(axis=1))
                        print()
                        print(f"Gradient Boosting Entire Dataset: {gb_entire_results}")

                    # Create empty DataFrame to store results
                    results_df = pd.DataFrame(columns=["N", "Norm", "Lambda", "Gamma", "K",
                                                       "LR_Entire_Time",
                                                       "Random Forest Entire Dataset",
                                                       "NN Entire Dataset",
                                                       "GP Entire Dataset",
                                                       "Gradient Boosting Entire Dataset"])

                    # Save the results as a DataFrame
                    results_df = pd.DataFrame({
                        "N": n,
                        "Norm": norm,
                        "Lambda": lam,
                        "Gamma": gam,
                        "K": k,
                        "LR_Entire_Time": lr_time,
                        "LR_Entire_MAE": lr_entire_results['MAE'],
                        "LR_Entire_RMSE": lr_entire_results['RMSE'],
                        "LR_Entire_MAD": lr_entire_results['MAD'],
                        "LR_Cluster_Time_Avg": pd.DataFrame(lr_cluster_times, index=[0]).mean(axis=1)[0],
                        "LR_Cluster_MAE_Avg": lr_cluster_results_pd.T['MAE'].mean(),
                        "LR_Cluster_RMSE_Avg": lr_cluster_results_pd.T['RMSE'].mean(),
                        "LR_Cluster_MAD_Avg": lr_cluster_results_pd.T['MAD'].mean(),
                        "LR_Cluster_MAE_Median": lr_cluster_results_pd.T['MAE'].median(),
                        "LR_Cluster_RMSE_Median": lr_cluster_results_pd.T['RMSE'].median(),
                        "LR_Cluster_MAD_Median": lr_cluster_results_pd.T['MAD'].median(),
                        "RF_Entire_Time": rf_time,
                        "RF_Entire_MAE": rf_entire_results['MAE'],
                        "RF_Entire_RMSE": rf_entire_results['RMSE'],
                        "RF_Entire_MAD": rf_entire_results['MAD'],
                        "RF_Cluster_Time_Avg": pd.DataFrame(rf_cluster_times, index=[0]).mean(axis=1)[0],
                        "RF_Cluster_MAE_Avg": rf_cluster_results_pd.T['MAE'].mean(),
                        "RF_Cluster_RMSE_Avg": rf_cluster_results_pd.T['RMSE'].mean(),
                        "RF_Cluster_MAD_Avg": rf_cluster_results_pd.T['MAD'].mean(),
                        "RF_Cluster_MAE_Median": rf_cluster_results_pd.T['MAE'].median(),
                        "RF_Cluster_RMSE_Median": rf_cluster_results_pd.T['RMSE'].median(),
                        "RF_Cluster_MAD_Median": rf_cluster_results_pd.T['MAD'].median(),
                        "NN_Entire_Time": nn_time,
                        "NN_Entire_MAE": nn_entire_results['MAE'],
                        "NN_Entire_RMSE": nn_entire_results['RMSE'],
                        "NN_Entire_MAD": nn_entire_results['MAD'],
                        "NN_Cluster_Time_Avg": pd.DataFrame(nn_cluster_times, index=[0]).mean(axis=1)[0],
                        "NN_Cluster_MAE_Avg": nn_cluster_results_pd.T['MAE'].mean(),
                        "NN_Cluster_RMSE_Avg": nn_cluster_results_pd.T['RMSE'].mean(),
                        "NN_Cluster_MAD_Avg": nn_cluster_results_pd.T['MAD'].mean(),
                        "NN_Cluster_MAE_Median": nn_cluster_results_pd.T['MAE'].median(),
                        "NN_Cluster_RMSE_Median": nn_cluster_results_pd.T['RMSE'].median(),
                        "NN_Cluster_MAD_Median": nn_cluster_results_pd.T['MAD'].median(),
                        "GP_Entire_Time": gp_time,
                        "GP_Entire_MAE": gp_entire_results['MAE'],
                        "GP_Entire_RMSE": gp_entire_results['RMSE'],
                        "GP_Entire_MAD": gp_entire_results['MAD'],
                        "GP_Cluster_Time_Avg": pd.DataFrame(gp_cluster_times, index=[0]).mean(axis=1)[0],
                        "GP_Cluster_MAE_Avg": gp_cluster_results_pd.T['MAE'].mean(),
                        "GP_Cluster_RMSE_Avg": gp_cluster_results_pd.T['RMSE'].mean(),
                        "GP_Cluster_MAD_Avg": gp_cluster_results_pd.T['MAD'].mean(),
                        "GP_Cluster_MAE_Median": gp_cluster_results_pd.T['MAE'].median(),
                        "GP_Cluster_RMSE_Median": gp_cluster_results_pd.T['RMSE'].median(),
                        "GP_Cluster_MAD_Median": gp_cluster_results_pd.T['MAD'].median(),
                        "GB_Entire_Time": gb_time,
                        "GB_Entire_MAE": gb_entire_results['MAE'],
                        "GB_Entire_RMSE": gb_entire_results['RMSE'],
                        "GB_Entire_MAD": gb_entire_results['MAD'],
                        "GB_Cluster_Time_Avg": pd.DataFrame(gb_cluster_times, index=[0]).mean(axis=1)[0],
                        "GB_Cluster_MAE_Avg": gb_cluster_results_pd.T['MAE'].mean(),
                        "GB_Cluster_RMSE_Avg": gb_cluster_results_pd.T['RMSE'].mean(),
                        "GB_Cluster_MAD_Avg": gb_cluster_results_pd.T['MAD'].mean(),
                        "GB_Cluster_MAE_Median": gb_cluster_results_pd.T['MAE'].median(),
                        "GB_Cluster_RMSE_Median": gb_cluster_results_pd.T['RMSE'].median(),
                        "GB_Cluster_MAD_Median": gb_cluster_results_pd.T['MAD'].median(),
                    }, index=[0])

                    # Save the results to a CSV file    
                    results_df.to_csv(f"results_{n}_{norm}_{lam}_{gam}_{k}.csv", index = False)

                    print()
                                        


N = 1000, Norm = 1, Lambda = 0.1, Gamma = 0.5, K = 3

Training time for entire dataset: 0.0009908676147460938

Linear Regression MSE for Entire Dataset: 0.10090998209080261

Linear Regression Training Times for Each Cluster:
Cluster 0.0: 0.000993967056274414
Cluster 1.0: 0.0007119178771972656
Cluster 2.0: 0.0006711483001708984

Linear Regression MSE for Each Cluster:
Cluster 0.0: 0.012722107247752275
Cluster 1.0: 0.0018078346424578835
Cluster 2.0: 0.2917196343294968

Average of all entries in lr_cluster_results:
MAE     0.085784
RMSE    0.231807
MAD     0.056430
dtype: float64

Linear Regression Entire Dataset: {'MAE': 0.07572404595198262, 'RMSE': 0.3176633156201745, 'MAD': 0.045674406410488144}
Training Time for Entire Dataset: 0.1434791088104248

Random Forest MSE for Entire Dataset: 0.0009692028865520469

Random Forest Training Times for Each Cluster:
Cluster 0.0: 0.06168985366821289
Cluster 1.0: 0.06725287437438965
Cluster 2.0: 0.06749677658081055

Random Forest MSE for Each Cluste

/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


NN Training Time for Entire Dataset: 0.10111594200134277

NN MSE for Entire Dataset: 0.09507812664001813

NN Training Times for Each Cluster:
Cluster 0.0: 0.06457805633544922
Cluster 1.0: 0.03092503547668457
Cluster 2.0: 0.051236867904663086

NN MSE for Each Cluster:
Cluster 0.0: 0.02055573186710895
Cluster 1.0: 0.007782743037333077
Cluster 2.0: 0.2790088465472565

Average of all entries in nn_cluster_results:
MAE     0.089187
RMSE    0.253268
MAD     0.045270
dtype: float64

NN Entire Dataset: {'MAE': 0.08746079921278742, 'RMSE': 0.3083474122479677, 'MAD': 0.038064733558556885}
GP Training Time for Entire Dataset: 0.25675010681152344

GP MSE for Entire Dataset: 0.07901941984891891

GP Training Times for Each Cluster:
Cluster 0.0: 0.06463909149169922
Cluster 1.0: 0.07325410842895508
Cluster 2.0: 0.06986808776855469

GP MSE for Each Cluster:
Cluster 0.0: 0.09015149623155594
Cluster 1.0: 0.00502082658931613
Cluster 2.0: 0.26126882433891296

Average of all entries in gp_cluster_results:
M

/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


NN Training Time for Entire Dataset: 0.13249778747558594

NN MSE for Entire Dataset: 0.09538576776716731

NN Training Times for Each Cluster:
Cluster 0.0: 0.030894041061401367
Cluster 1.0: 0.03580880165100098
Cluster 2.0: 0.012879133224487305
Cluster 3.0: 0.009495973587036133
Cluster 4.0: 0.02063894271850586
Cluster 5.0: 0.026911020278930664

NN MSE for Each Cluster:
Cluster 0.0: 0.004788359096111033
Cluster 1.0: 0.01083499392500426
Cluster 2.0: 0.0061710333110069765
Cluster 3.0: 1.2313614408890652
Cluster 4.0: 0.01911571446891896
Cluster 5.0: 0.3745712618357043

Average of all entries in nn_cluster_results:
MAE     0.128237
RMSE    0.351966
MAD     0.049290
dtype: float64

NN Entire Dataset: {'MAE': 0.09771067465772702, 'RMSE': 0.30884586409270126, 'MAD': 0.044649708875513186}
GP Training Time for Entire Dataset: 0.24770832061767578

GP MSE for Entire Dataset: 0.07901941984891891

GP Training Times for Each Cluster:
Cluster 0.0: 0.07380080223083496
Cluster 1.0: 0.10318183898925781
Clu

...

/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


NN Training Time for Entire Dataset: 1.8948681354522705

NN MSE for Entire Dataset: 0.08997616063550794

NN Training Times for Each Cluster:
Cluster 0.0: 0.2546720504760742
Cluster 1.0: 0.12215590476989746
Cluster 2.0: 0.271543025970459
Cluster 3.0: 0.2690308094024658
Cluster 4.0: 0.08418989181518555
Cluster 5.0: 0.05862307548522949

NN MSE for Each Cluster:
Cluster 0.0: 0.06510874213516042
Cluster 1.0: 0.22846653200249906
Cluster 2.0: 0.027926060624279443
Cluster 3.0: 0.0679570015233908
Cluster 4.0: 0.12668289984187125
Cluster 5.0: 0.016276510409034262

Average of all entries in nn_cluster_results:
MAE     0.078471
RMSE    0.274075
MAD     0.032228
dtype: float64

NN Entire Dataset: {'MAE': 0.07370750577684347, 'RMSE': 0.2999602650944087, 'MAD': 0.02384816191490488}
GP Training Time for Entire Dataset: 9.233605146408081

GP MSE for Entire Dataset: 0.06504029780626297

GP Training Times for Each Cluster:
Cluster 0.0: 1.3206112384796143
Cluster 1.0: 1.3362438678741455
Cluster 2.0: 0.300

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/ekinokos2/anaconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/ekinokos2/anaconda3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Training time for entire dataset: 0.5943160057067871

Gradient Boosting MSE for Entire Dataset: 0.03812300038865588

Gradient Boosting Training Times for Each Cluster:
Cluster 0.0: 0.14795589447021484
Cluster 1.0: 0.15158796310424805
Cluster 2.0: 0.07992100715637207
Cluster 3.0: 0.06630969047546387
Cluster 4.0: 0.04640483856201172
Cluster 5.0: 0.06294727325439453

Gradient Boosting MSE for Each Cluster:
Cluster 0.0: 0.06274962066655296
Cluster 1.0: 0.03665499167251396
Cluster 2.0: 0.16071195685049947
Cluster 3.0: 0.023243791335450972
Cluster 4.0: 0.18412811294053802
Cluster 5.0: 0.011363681478177559

Average of all entries in gb_cluster_results:
MAE     0.052038
RMSE    0.255167
MAD     0.019330
dtype: float64

Gradient Boosting Entire Dataset: {'MAE': 0.046026849144372294, 'RMSE': 0.19525112135057224, 'MAD': 0.022916511687969987}

N = 10000, Norm = 1, Lambda = 0.1, Gamma = 0.5, K = 9



KeyboardInterrupt: 

In [58]:
# convert lr_cluster_times to a DataFrame
pd.DataFrame(lr_cluster_times, index=[0]).mean(axis=1)

0    0.000942
dtype: float64

In [63]:
lr_cluster_results_pd.T['MAE'].median()

0.08853538707805389

In [38]:
lr_entire_results['MAE']

0.07572404595198262

In [36]:
# Convert lr_entire_results to a DataFrame
pd.DataFrame(lr_entire_results, index=[0]).rename(columns={"MAE": "LR_Entire_MAE", "RMSE": "LR_Entire_RMSE", "MAD": "LR_Entire_MAD"})

,LR_Entire_MAE,LR_Entire_RMSE,LR_Entire_MAD
0,0.075724,0.317663,0.045674


In [ ]:
results = {
                        "N": n,
                        "Norm": norm,
                        "Lambda": lam,
                        "Gamma": gam,
                        "K": k,
                        "Linear Regression": {
                            "Training Time": lr_time,
                            "MSE Entire Dataset": lr_entire_mse,
                            "MSE Per Cluster": linear_regression_mse,
                            "Cluster Training Times": lr_cluster_times,
                            "Cluster Results": lr_cluster_results,
                            "Entire Dataset Results": lr_entire_results
                        },
                        "Random Forest": {
                            "Training Time": rf_time,
                            "MSE Entire Dataset": rf_entire_mse,
                            "MSE Per Cluster": random_forest_mse,
                            "Cluster Training Times": rf_cluster_times,
                            "Cluster Results": rf_cluster_results,
                            "Entire Dataset Results": rf_entire_results
                        },
                        "Neural Network": {
                            "Training Time": nn_time,
                            "MSE Entire Dataset": nn_mse,
                            "MSE Per Cluster": neural_network_mse,
                            "Cluster Training Times": nn_cluster_times,
                            "Cluster Results": nn_cluster_results,
                            "Entire Dataset Results": nn_entire_results
                        },
                        "Gaussian Process": {
                            "Training Time": gp_time,
                            "MSE Entire Dataset": gp_entire_mse,
                            "MSE Per Cluster": gaussian_process_mse,
                            "Cluster Training Times": gp_cluster_times,
                            "Cluster Results": gp_cluster_results,
                            "Entire Dataset Results": gp_entire_results
                        },
                        "Gradient Boosting": {
                            "Training Time": gb_time,
                            "MSE Entire Dataset": gb_entire_mse,
                            "MSE Per Cluster": gradient_boosting_mse,
                            "Cluster Training Times": gb_cluster_times,
                            "Cluster Results": gb_cluster_results,
                            "Entire Dataset Results": gb_entire_results
                        }
                    }

                    # Save the results to a file
                    with open("sensitivity_results.txt", "a") as f:
                        f.write(str(results))
                        f.write("\n")
                        f.write("\n")

In [27]:
results

{'N': 1000,
 'Norm': 1,
 'Lambda': 0.1,
 'Gamma': 0.5,
 'K': 3,
 'Linear Regression': {'Training Time': 0.0008699893951416016,
  'MSE Entire Dataset': 0.10090998209080261,
  'MSE Per Cluster': {0.0: 0.012722107247752275,
   1.0: 0.0018078346424578835,
   2.0: 0.2917196343294968},
  'Cluster Training Times': {0.0: 0.0010688304901123047,
   1.0: 0.0011599063873291016,
   2.0: 0.0009622573852539062},
  'Cluster Results': {0.0: {'MAE': 0.08853538707805389,
    'RMSE': 0.1127923190990959,
    'MAD': 0.07357154908074431},
   1.0: {'MAE': 0.03035321666698034,
    'RMSE': 0.04251863876534482,
    'MAD': 0.025996888081637914},
   2.0: {'MAE': 0.1384634792018459,
    'RMSE': 0.5401107611680189,
    'MAD': 0.06972040358351161}},
  'Entire Dataset Results': {'MAE': 0.07572404595198262,
   'RMSE': 0.3176633156201745,
   'MAD': 0.045674406410488144}},
 'Random Forest': {'Training Time': 0.19759082794189453,
  'MSE Entire Dataset': 0.0009520665756600807,
  'MSE Per Cluster': {0.0: 0.00255930852078014

In [22]:
# Load the results as a dictionary
results = {}
with open("sensitivity_results.txt", "r") as f:
    for line in f:
        line = line.strip()
        if line:
            line = line.replace("'", "\"")
            line = line.replace("True", "true")
            line = line.replace("False", "false")
            line = line.replace("None", "null")
            line = line.replace("nan", "null")
            line = line.replace("inf", "null")
            line = line.replace("-inf", "null")
            line = line.replace("array", "null")
            line = line.replace("dtype", "null")
            line = line.replace("float64", "null")
            line = line.replace("int64", "null")
            line = line.replace("object", "null")
            line = line.replace("int32", "null")
            line = line.replace("float32", "null")
            line = line.replace("int16", "null")
            line = line.replace("float16", "null")
            line = line.replace("int8", "null")
            line = line.replace("float8", "null")
            line = line.replace("int4", "null")
            line = line.replace("float4", "null")
            line = line.replace("int2", "null")
            line = line.replace("float2", "null")
            line = line.replace("int1", "null")
            line = line.replace("float1", "null")
            line = line.replace("int0", "null")
            line = line.replace("float0", "null")
            line = line.replace("int", "null")
            line = line.replace("float", "null")
            line = line.replace("null", "null")
            line = line.replace(":", ": ")
            line = line.replace(",", ", ")
            line = line.replace("{", "{ ")
            line = line.replace("}", " }")
            line = line.replace("[", "[ ")
            line = line.replace("]", " ]")
            line = line.replace("  ", " ")

            line = line.replace("Training Time", "\"Training Time\"")
            line = line.replace("MSE Entire Dataset", "\"MSE Entire Dataset\"")
            line = line.replace("MSE Per Cluster", "\"MSE Per Cluster\"



In [24]:
results

{}